In [ ]:
import torch
import torch.nn as nn
import torchaudio

from audio import get_clean_tensor,get_crunch_tensor,get_distortion_tensor,normalize_tensor
from utilities import plot_waveform

dry = get_clean_tensor()
dry = normalize_tensor(dry)
# plot_waveform(dry)
crunch = get_crunch_tensor()
# plot_waveform(crunch)
distortion = get_distortion_tensor()
# plot_waveform(distortion)

data_minutes = 5
test_split_ratio = 0.2

train_time_seconds = int(data_minutes * (1 - test_split_ratio) * 60)
val_time_seconds = int(data_minutes * test_split_ratio * 60)
train_samples = int(44_100 * train_time_seconds)
val_samples = int(44_100 * val_time_seconds)

x = dry[0]
y = crunch[0]

In [ ]:
from sklearn.model_selection import KFold

In [ ]:
x_train = x[:train_samples]
y_train = y[:train_samples]

x_val = x[train_samples:train_samples+val_samples]
y_val = y[train_samples:train_samples+val_samples]

x_train = x_train.view(-1,200,1)
y_train = y_train.view(-1,200,1)

In [ ]:
from torch.utils.data import TensorDataset,DataLoader

train_merged = TensorDataset(x_train,y_train)

train_dataloader = DataLoader(train_merged, batch_size=100, shuffle=False)

In [ ]:

for batch_ndx, sample in enumerate(train_dataloader):
    xx = sample[0]
    print(xx.shape)
    break
    

In [ ]:
print(x_train.shape)
print(y_train.shape)
print(x_val.shape)
print(y_val.shape)

In [ ]:
from train import train
from models import LSTM

In [ ]:
model = LSTM(n_hidden=64,n_layers=1)
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

In [ ]:
model = train(model,optimizer,x_train,y_train,x_val,y_val,10,100,200,64,'cuda','temporary/temp','temporary/')

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
batch_sizes = [100,200,300,400,500]
steps_nums = [100,200,300]
epochs = 25

for sn in steps_nums:
    for bs in batch_sizes:
        if (bs == 400 and sn == 300) or (bs == 500 and sn > 100):
            continue

        path = 'scores/batch_sizes/bs_' + str(bs) + '_ns_' + str(sn) + '.npy'
        scores = np.load(path)
        tl = scores[:,0]
        plt.plot(range(epochs), tl,label=f'bs={bs}, ns={sn}')
    plt.title(f'train loss, num_steps={sn}')
    plt.legend()
    plt.savefig(f'plots/train_loss_num_steps_{sn}.png')
    # plt.draw()
    plt.pause(0.05)

In [ ]:
batch_sizes = [100,200,300,400,500]
steps_nums = [100,200,300]
epochs = 25

for sn in steps_nums:
    for bs in batch_sizes:
        if (bs == 400 and sn == 300) or (bs == 500 and sn > 100):
            continue

        path = 'scores/batch_sizes/bs_' + str(bs) + '_ns_' + str(sn) + '.npy'
        scores = np.load(path)
        vl = scores[:,1]
        plt.plot(range(epochs), vl,label=f'bs={bs}, ns={sn}')
    plt.title(f'test loss, num_steps={sn}')
    plt.legend()
    plt.savefig(f'plots/test_loss_num_steps_{sn}.png')
    # plt.draw()
    plt.pause(0.05)

In [ ]:
layers = [1,2]
hidden_sizes = [64,128,256]
epochs = 25

for layer in layers:
    for hidden_size in hidden_sizes:
        lstm_path = 'scores/grid_search/gs_lstm_' + str(layer) + '_' + str(hidden_size) + '.npy'
        sc_path = 'scores/grid_search/gs_sc_' + str(layer) + '_' + str(hidden_size) + '.npy'
        dc_path = 'scores/grid_search/gs_dc_' + str(layer) + '_' + str(hidden_size) + '.npy'
        lstm_scores = np.load(lstm_path)
        sc_scores = np.load(sc_path)
        dc_scores = np.load(dc_path)
        lstm_tl = lstm_scores[:,0]
        sc_tl = sc_scores[:,0]
        dc_tl = dc_scores[:,0]
        plt.plot(range(epochs), lstm_tl,label='lstm')
        plt.plot(range(epochs), sc_tl,label='singleConv')
        plt.plot(range(epochs), dc_tl,label='doubleConv')
        plt.title(f'train loss, layers={layer},hidden_size={hidden_size}')
        plt.legend()
        plt.savefig(f'plots/train_loss_gs_{layer}_{hidden_size}.png')
        plt.draw()
        plt.pause(0.05)

In [ ]:
layers = [1,2]
hidden_sizes = [64,128,256]
epochs = 25

for layer in layers:
    for hidden_size in hidden_sizes:
        lstm_path = 'scores/grid_search/gs_lstm_' + str(layer) + '_' + str(hidden_size) + '.npy'
        sc_path = 'scores/grid_search/gs_sc_' + str(layer) + '_' + str(hidden_size) + '.npy'
        dc_path = 'scores/grid_search/gs_dc_' + str(layer) + '_' + str(hidden_size) + '.npy'
        lstm_scores = np.load(lstm_path)
        sc_scores = np.load(sc_path)
        dc_scores = np.load(dc_path)
        lstm_tl = lstm_scores[:,1]
        sc_tl = sc_scores[:,1]
        dc_tl = dc_scores[:,1]
        plt.plot(range(epochs), lstm_tl,label='lstm')
        plt.plot(range(epochs), sc_tl,label='singleConv')
        plt.plot(range(epochs), dc_tl,label='doubleConv')
        plt.title(f'test loss, layers={layer},hidden_size={hidden_size}')
        plt.legend()
        plt.savefig(f'plots/test_loss_gs_{layer}_{hidden_size}.png')
        plt.draw()
        plt.pause(0.05)

In [ ]:
layers = [1,2]
hidden_sizes = [64,128,256]
epochs = 25
lstm_mean_times = {}
sc_mean_times = {}
dc_mean_times = {}
for layer in layers:
    for hidden_size in hidden_sizes:
        lstm_path = 'scores/grid_search/gs_lstm_' + str(layer) + '_' + str(hidden_size) + '.npy'
        sc_path = 'scores/grid_search/gs_sc_' + str(layer) + '_' + str(hidden_size) + '.npy'
        dc_path = 'scores/grid_search/gs_dc_' + str(layer) + '_' + str(hidden_size) + '.npy'
        lstm_scores = np.load(lstm_path)
        sc_scores = np.load(sc_path)
        dc_scores = np.load(dc_path)
        lstm_mt = lstm_scores[:,2]
        sc_mt = sc_scores[:,2]
        dc_mt = dc_scores[:,2]

        lstm_mean_times.update({f'layer={layer} hidden={hidden_size}':np.mean(lstm_mt)})
        sc_mean_times.update({f'layer={layer} hidden={hidden_size}':np.mean(sc_mt)})
        dc_mean_times.update({f'layer={layer} hidden={hidden_size}':np.mean(dc_mt)})

print(lstm_mean_times)
print(sc_mean_times)
print(dc_mean_times)



In [ ]:
x = range(6)

y = list(lstm_mean_times.values())
plt.scatter(x,y)

for i,v in enumerate(list(lstm_mean_times.keys())):
        plt.annotate('lstm ' + v,(x[i],y[i]))
plt.title('lstm mean time per epoch [s]')
# plt.savefig('plots/bs_tpe.png')
plt.draw()
plt.pause(0.05)

In [ ]:
y = list(sc_mean_times.values())
# plt.bar(x,y)
plt.scatter(x,y)

for i,v in enumerate(list(sc_mean_times.keys())):
        plt.annotate('sc ' + v,(x[i],y[i]))
plt.title('singleConv mean time per epoch [s]')
# plt.savefig('plots/bs_tpe.png')
plt.draw()
plt.pause(0.05)

In [ ]:
y = list(dc_mean_times.values())
plt.scatter(x,y)

for i,v in enumerate(list(dc_mean_times.keys())):
        plt.annotate('dc ' + v,(x[i],y[i]))
plt.title('doubleConv mean time per epoch [s]')
# plt.savefig('plots/bs_tpe.png')
plt.draw()
plt.pause(0.05)

In [ ]:
batch_sizes = [100,200,300,400,500]
steps_nums = [100,200,300]
epochs = 25
mean_times = {}
for sn in steps_nums:
    for bs in batch_sizes:
        if (bs == 400 and sn == 300) or (bs == 500 and sn > 100):
            continue

        path = 'scores/batch_sizes/bs_' + str(bs) + '_ns_' + str(sn) + '.npy'
        scores = np.load(path)
        et = scores[:,2]
        mean_times.update({f'ns={sn} bs={bs}':np.mean(et)})

# print(mean_times)
x = range(12)
y = list(mean_times.values())
# plt.bar(x,y)
plt.scatter(x,y)

for i,v in enumerate(list(mean_times.keys())):
        plt.annotate(v,(x[i],y[i]))
plt.title('mean time per epoch [s]')
plt.savefig('plots/bs_tpe.png')
plt.pause(0.05)